In [2]:
import pandas as pd
import numpy as np
import os

In [72]:
def get_diff_bin(data):
    average_bin, bin_0, bin_24 = [], [], []

    for row in range(24, data.shape[0], 25):
        bin_24.append(data.loc[row, 'bin_volume'])

    for row in range(0, data.shape[0], 25):
        bin_0.append(data.loc[row, 'bin_volume'])
        average_bin.append(data.loc[row+ 1: row+ 24, 'daily_volume'].mean())
    return average_bin, bin_0, bin_24


def transformer(data):
    transformed_nums = [-1 if num < 0 else 1 if num > 0 else 0 for num in data]
    return transformed_nums

average_bin, bin_0, bin_24 = get_diff_bin(data_46)
# print(len(average_bin), len(bin_0), len(bin_24))

diff_average_bin = transformer([(average_bin[i] - average_bin[i-1]) for i in range(1, len(average_bin))])
diff_bin_0_24 = [bin_0[i]- bin_24[i] for i in range(len(bin_24))][1:]
diff_bin_0_0 = [(bin_0[i]- bin_0[i-1])/bin_0[i-1] for i in range(1, len(bin_0))]
# print(len(diff_average_bin), len(diff_bin_0_0), len(diff_bin_0_24))

data_diff = pd.DataFrame({'ddiff_average_bin': diff_average_bin, 'bin_0_0': diff_bin_0_0, 'bin_0_24': diff_bin_0_24})
# data_diff.corr()

In [31]:
def df2matrix(file_path:str, col_name: str=None, bin_num: bool= True):
    """
    转换数据结构，以date为横坐标，制定col_name为纵坐标
    """

    df = pd.read_csv(file_path)
    df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
    df.set_index('date', inplace= True)
    df.sort_index(inplace= True)
    n, k = df.shape[0], 25
    # TODO: 1、把 bin0 分离出来，而后补充到特征里面去，也就是需要在  self.genNewFeatureBinVolume() 把bin0 特征融合进来
    if bin_num:
        data = pd.DataFrame(np.array(df[col_name]).reshape(int(n/k), k),
                                columns= ['bin{}'.format(i) for i in range(k)]).drop('bin0', axis=1)
    else:
        bin0_list = []
        for i in range(0, df.shape[0], 25):
            if i == 0:
                bin0_list.append([(df[col_name].iloc[i]- df[col_name].iloc[i])/ df[col_name].iloc[i]])
            else:
                bin0_list.append([(df[col_name].iloc[i]- df[col_name].iloc[i-25])/ df[col_name].iloc[i-25]])
        result = np.array([[element[0]]*(k-1) for element in bin0_list])
        print(result.shape)
        data = pd.DataFrame(np.array(result),
                            columns= ['bin0' for i in range(k-1)])
    data['date'] = pd.to_datetime(list(sorted(set(df.index))), format='%Y/%m/%d')
    data.set_index('date', inplace= True)
    data.sort_index(inplace=True)
    return data

data = df2matrix(file_path= '../data/0308/000046_XSHE_25_daily.csv', col_name= 'bin_volume', bin_num= False)

(197, 24)


In [19]:
import pandas as pd

df = pd.read_csv('../data/comment/000046.csv')
df2 = pd.read_csv('../data/0308/000046_XSHE_25_daily.csv')
# df2['date'] = pd.to_datetime(df2['date'])
# df2['date'] = df2['date'].dt.date
df['publishDate'] = pd.to_datetime(df['publishDate'])

start_time = '2020-5-12 15:00'
end_time = '2021-3-8 15:00'
filtered_df = df[(df['publishDate'] >= start_time) & (df['publishDate'] <= end_time)].copy().reset_index(drop=True)
filtered_df['dates'] = filtered_df['publishDate'].dt.date

In [20]:
filtered_df.head()

,post_id,publishDate,newsTitle,posterId,posterName,dates
0,929174946,2020-05-12 15:07:40,泛海下跌周期太长，被套资本太多，没有啥机会暴涨的，就像上次一样涨到8元就被打熄火,5323335728465078,股友oRx5gP,2020-05-12
1,929207943,2020-05-12 17:26:28,民生证券将挥别北京 泛海控股再度布局上海滩,4506395531315256,投资者攻略,2020-05-12
2,929210791,2020-05-12 17:39:29,适合在家创业的6个项目，有适合您的吗？,7301305858084640,小息谈资讯,2020-05-12
3,929323847,2020-05-13 06:01:54,泛海，真的跌得好惨！,9273094111979056,水中月7878,2020-05-13
4,929331305,2020-05-13 06:49:52,看看大盘走的多好，震荡上行，好多个股都跌惨了，什么破股市啊,3407145336565478,魅力的林顺昊,2020-05-13


In [27]:
from datetime import date

# 创建一个日期对象
date_obj = date(2020, 5, 12)

# 手动格式化日期为 "YYYY-M-D" 格式
formatted_date_custom = f"{date_obj.year}-{date_obj.month}-{date_obj.day}"
print(formatted_date_custom)

2020-5-12


In [ ]:
date_XSHE = set(df2['date'])
date_XSHE.add(str(filtered_df.loc[0, 'dates']))
date_XSHE

In [31]:
def calculate_time_bins(day):
    bins = {}
    # bin_0: 从前一天的15:00到当天的09:30
    bin_0_start = f'{day - pd.Timedelta(days=1)} 15:00'
    bin_0_end = f'{day} 09:30'
    bins['bin0'] = [bin_0_start, bin_0_end]

    # 初始化bin_1的起始时间为bin_0的结束时间
    current_start = bin_0_end

    # 循环生成剩下的23个bin
    for i in range(1, 25):
        if i == 10:
            bins[f'bin{i}'] = [current_start, f'{day} 11:30']
            current_start = f'{day} 11:30'
        elif i == 11:
            bins[f'bin{i}'] = [f'{day} 11:30', f'{day} 13:00']
            current_start = f'{day} 13:00'
        elif i == 24:
            bins[f'bin{i}'] = [current_start, f'{day} 15:00']
        else:
            current_end = pd.to_datetime(current_start) + pd.Timedelta(minutes=9.875)
            bins[f'bin{i}'] = [current_start, current_end.strftime('%Y-%m-%d %H:%M')]
            current_start = current_end.strftime('%Y-%m-%d %H:%M')
    return bins

times_set = set()
for i in filtered_df['dates']:
    times_set.add(i)

data_bin = pd.DataFrame(index = list(date_XSHE),
                        columns= [f'bin{i}' for i in range(0, 25)])
data_bin.index = pd.to_datetime(data_bin.index)
data_bin = data_bin.sort_index(ascending= True)
for day in data_bin.index[1:]:
    # print()
    if str(day.date()) in date_XSHE:
        bin_newstitle = {}
        bins = calculate_time_bins(day.date())
        for bin in bins:
            bin_newstitle[bin] = filtered_df[(filtered_df['publishDate'] >= bins[bin][0]) & (filtered_df['publishDate'] <= bins[bin][1])].loc[:, 'newsTitle'].values
        data_bin.loc[day] = bin_newstitle

In [41]:
from zhipuai import ZhipuAI
import pandas as pd


def prompt_build():
    prompt = f"作为情感分析专家，请根据以下输入文本判断评论的情感态度。\
        要求:\
            1.情感态度表示方式：消极:-1，积极:1，中性:0\
            2.请直接返回对应的数字，无需提供额外解释\
        示例:'这只股票要完了'.输出:-1\
        请分析以下文本的情感态度:"
    return prompt

def ZhiPuSentiment(input_comment: str,
                api_key: str= 'daadb0e4e98a27cb82436c0f321eeb53.GtyK1RkdCkzhxt4V'):
    client = ZhipuAI(
        api_key= api_key
    )
    prompt = prompt_build()
    response = client.chat.completions.create(
        model = "glm-3-turbo",
        messages= [
            {'role': "user", "content": prompt+ input_comment},
        ],
        temperature = 0.3,
    )
    # print(response.choices[0].message.content, input_comment)
    sentiment_scores = []
    print(response.choices[0].message.content.replace(' ', '').split(','))
    for score in  response.choices[0].message.content.replace(' ', '').split(','):
        try:
            sentiment_scores.append(int(score))
        except Exception as e:
            print(e)
    # print(sentiment_scores)
    score = max(sentiment_scores, key= sentiment_scores.count)
    return score


df = pd.read_csv('../data/0308/0308-comment/000046_comment.csv', index_col='Unnamed: 0')
df2 = pd.read_csv('../data/0308/0303-number/000046_comment_sentiment.csv', index_col='Unnamed: 0')
i, j = 19, 0
print(df.iloc[i, j], '\n', df2.iloc[i, j])
ZhiPuSentiment(input_comment= df.iloc[i, j])

['泛海不用借壳只用整体上市及少付利息就可做到利润80亿到一百多亿的混合金融，投资及'
 '很早捐几十亿的，投资民生银行，海通，蚂蚁等几十独角兽的，二十年囤核心城市核心地的' '民生证券过会8单' '明天应该要收复3.91'
 '混合金融业绩一到三元角股是目标，分红毎年一到五角是目标，最终比卢总强的企业家没几'
 '1月14日，503涨停，我卖来马上反悔了，马上挂单买怎么也买不到，15年我20融' '这两天有点强[赞]'
 '中国人节俭爱投资创业，最终金融全牌照几十独角兽盈利能力会超九成快消企业及95％的' '弱反弹而已到4.5再说'
 '根据中央商务引战46亿时仔细评估升值378亿后，中美土地物业房地产涨了不少，蚂蚁'
 '30岁的经理喜欢高价呼人的，其实低价值值股业绩股分红股只要涨一点点差价比例大，数'] 
 -1
['-1', '0', '0', '0', '0', '-1', '-1', '0', '-1', '-1', '0', '-1', '-1', '0', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1',

-1

In [53]:
df = pd.read_csv('../data/0308/0308-comment/000046_comment.csv')
print(df.iloc[0, 1])
# for i in df.iloc[0,1].replace("[", '').replace("]",'').strip().split(' '):
# for i in df.iloc[0, 1].replace("[", '').replace("]",''):
    # print(i)

['泛海下跌周期太长，被套资本太多，没有啥机会暴涨的，就像上次一样涨到8元就被打熄火' '民生证券将挥别北京 泛海控股再度布局上海滩'
 '适合在家创业的6个项目，有适合您的吗？' '泛海，真的跌得好惨！' '看看大盘走的多好，震荡上行，好多个股都跌惨了，什么破股市啊'
 '泛海控股05月12日获深股通减仓27.45万股']


In [57]:
a = '../data/0308/0308-comment/000998_comment.csv'
a[26:len(a)-4]

'000998_comment'

In [156]:
bins

{'bin0': ['2021-03-07 15:00', '2021-03-08 09:30'],
 'bin1': ['2021-03-08 09:30', '2021-03-08 09:39'],
 'bin2': ['2021-03-08 09:39', '2021-03-08 09:48'],
 'bin3': ['2021-03-08 09:48', '2021-03-08 09:57'],
 'bin4': ['2021-03-08 09:57', '2021-03-08 10:06'],
 'bin5': ['2021-03-08 10:06', '2021-03-08 10:15'],
 'bin6': ['2021-03-08 10:15', '2021-03-08 10:24'],
 'bin7': ['2021-03-08 10:24', '2021-03-08 10:33'],
 'bin8': ['2021-03-08 10:33', '2021-03-08 10:42'],
 'bin9': ['2021-03-08 10:42', '2021-03-08 10:51'],
 'bin10': ['2021-03-08 10:51', '2021-03-08 11:30'],
 'bin11': ['2021-03-08 11:30', '2021-03-08 13:00'],
 'bin12': ['2021-03-08 13:00', '2021-03-08 13:09'],
 'bin13': ['2021-03-08 13:09', '2021-03-08 13:18'],
 'bin14': ['2021-03-08 13:18', '2021-03-08 13:27'],
 'bin15': ['2021-03-08 13:27', '2021-03-08 13:36'],
 'bin16': ['2021-03-08 13:36', '2021-03-08 13:45'],
 'bin17': ['2021-03-08 13:45', '2021-03-08 13:54'],
 'bin18': ['2021-03-08 13:54', '2021-03-08 14:03'],
 'bin19': ['2021-03-08